In [ ]:
#Read both csv files. You can use “pandas” or “polars” libraries to read it as dataframe in Python.

In [1]:
!pip install polars

In [2]:
!pip install scikit-learn

In [3]:
import polars as pl
import re
from sklearn import preprocessing
import numpy as np

In [4]:
train_file = pl.read_csv("train.csv")
test_file = pl.read_csv("test.csv")

In [3]:
#For each review, remove all <br /> tokens. You can use “re” library in Python that uses regular expressions
#or you can use any other library of your choice that allow string removl/replacement.
#and
# Change labels from text to integers (0 for negative and 1 for positive).
# You can do it directly in pandas dataframe or use scikit-learn label encoder.

In [5]:
pattern = r'(<br />)' #() was the solution bro..... [] :(
# Replace all occurrences of character s with an empty string
#For the train csv file
updated_dict = {}
review_list=[]
sentiment_list = []
for row in range(0,2000):    
    string = train_file.row(row,named = True) #reads the data in each row, and convert it in dictionary.
    mod_string = re.sub(pattern,'',string['review']) #relace the word <br /> with ''>
    string['review'] = mod_string #the review will be updated with removed word.
    review_list.append(mod_string) #that review will be appended in a list of reviews
    sentiment_list.append(string['sentiment'])    #whatever the sentiment is for that review, it will aslo be appended 

le = preprocessing.LabelEncoder()
le.fit(sentiment_list)
labeled_list1 = list(le.transform(sentiment_list))

updated_dict['review'] = review_list
updated_dict['sentiment'] = labeled_list1 # 0  and 1 representation of the labels.

modified_data = pl.DataFrame(updated_dict)
modified_data.write_csv("modified_train.csv")

In [6]:
pattern = r'(<br />)' #() was the solution bro..... [] :(
# Replace all occurrences of character s with an empty string
#For the train csv file
updated_dict2 = {} #for test
review_list2=[] #for test
sentiment_list2 = [] # for test
for row in range(0,500):    
    string2 = test_file.row(row,named = True) #reads the data in each row, and convert it in dictionary.
    mod_string2 = re.sub(pattern,'',string2['review']) #relace the word <br /> with ''>
    string2['review'] = mod_string2 #the review will be updated with removed word.
    review_list2.append(mod_string2) #that review will be appended in a list of reviews
    sentiment_list2.append(string2['sentiment']) #whatever the sentiment is for that review, it will aslo be appended 

le = preprocessing.LabelEncoder()
le.fit(sentiment_list2)
labeled_list = list(le.transform(sentiment_list2))
updated_dict2['review'] = review_list2
updated_dict2['sentiment'] = labeled_list #Converted the labels to 0(negative) and 1(positive) 

modified_data2 = pl.DataFrame(updated_dict2) #converts dictionary in to a dataframe
modified_data2.write_csv("modified_test.csv") #writes it into a csv file.

In [7]:
# Separate out labels and texts. You should have train texts, train labels, test texts, 
# and test labels in separate variables.
# split into train test sets
# from sklearn.model_selection import train_test_split
# train_file2 = pl.read_csv("modified_train.csv")
# train_texts, train_labels = train_test_split(train_file2)
train_texts = review_list
train_labels = labeled_list1
test_texts = review_list2
test_labels = labeled_list

In [84]:
#For the preprocessed texts above, convert them to vectors based on frequency 
#(counts) using scikit-learn. Do this for both train and test splits. Remember, you 
#can only use train texts to determine the vocabulary of the dataset. Hint: You 
#will have one vectorizer that has seen training data and will be used to transform 
#the testing data.
len(test_labels)

500

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# To create a Count Vectorizer, we simply need to instantiate one.
# There are special parameters we can set here when making the vectorizer, but
# for the most basic example, it is not needed.
vectorizer = CountVectorizer(binary = True)

# To actually create the vectorizer, we simply need to call fit on the text
# data that we wish to fix
train_vector = vectorizer.fit_transform(train_texts).toarray()
test_vector = vectorizer.transform(test_texts).toarray()


# print(train_vector.shape, test_vector.shape)
# vd_train = pl.DataFrame(data=train_array,schema = vectorizer.get_feature_names())
# vd_train.write_csv("counterized_train.csv") #writes it into a csv file.
# print(vd_train)

In [10]:
#sklearn.feature_selection.mutual_info_classif(X, y, *, discrete_features='auto', n_neighbors=3, copy=True, random_state=None)
from sklearn.feature_selection import mutual_info_classif
info_gain = mutual_info_classif(train_vector,train_labels,random_state = 1)

In [11]:
len(info_gain)

25235

In [12]:
# fd = pl.DataFrame(data=info_gain,schema = vectorizer.get_feature_names())
features = vectorizer.get_feature_names()

C:\Users\HPdemo\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
len(features)

25235

In [14]:
data_dict = {}
for val in range(25235):
    data_dict[features[val]] = info_gain[val]

In [15]:
sorted_dict_data = sorted(data_dict.items(), key = lambda kv: kv[1])

In [16]:
for it in range(10):
    print(sorted_dict_data[it])
#Bottom Ten features with info gain value 0

('00', 0.0)
('000', 0.0)
('04', 0.0)
('07', 0.0)
('10', 0.0)
('101', 0.0)
('102', 0.0)
('103', 0.0)
('10the', 0.0)
('10this', 0.0)


In [17]:
top_ten_features= []
for it2 in range(-11,0,1):
    top_ten_features.append(sorted_dict_data[it2])
print(top_ten_features[::-1])
#Top 10 features with info gain greater than 0.

[('obtrusive', 0.04852586544958659), ('bad', 0.04233031764907125), ('paleontologist', 0.042286314254307955), ('toothpaste', 0.0421768284270978), ('uribe', 0.041171971141280794), ('lucas', 0.04083139728660079), ('awful', 0.040299617256842835), ('cultivation', 0.03969472814433961), ('burr', 0.039481084137305666), ('excellent', 0.038772235612442696), ('somehow', 0.038439951385150994)]


In [41]:
np.shape(train_array) #TRAIN AND TEST SHAPES NOT SAME.


(2000, 25235)

In [42]:
np.shape(test_array) #TRAIN AND TEST SHAPES DON;T MATHC :(

(500, 12366)

In [43]:
train_labels[:1]

[1]

In [49]:
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_selection import SelectPercentile as SP
from sklearn.tree import DecisionTreeClassifier as DTC
selector = SP(score_func=mutual_info_classif,percentile=49.005) # #49.005 Matches the X of decision tree. Features.
selector.fit(train_vector,train_labels)
X_reduced = selector.transform(train_vector)
X_test_reduced = selector.transform(test_vector)
model_4 = DTC(criterion="entropy").fit(X_reduced,train_labels)

In [50]:
X_test_reduced.shape, np.array(test_labels).shape, test_vector.shape

((500, 12366), (500,), (500, 25235))

In [69]:
score_4 = model_4.score(X_test_reduced,np.array(test_labels)) #Model_4/score_4 is for reduced features.
print(f"Score = {score_4}")

Score = 0.702


In [70]:
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_selection import SelectPercentile as SP
from sklearn.tree import DecisionTreeClassifier as DTC

model_1 = DTC(criterion="entropy").fit(train_vector,train_labels)
score_1 = model_1.score(test_vector,test_labels)
print(f"score_1:{score_1}") #Model_1 and score_1 is fpr whole dataset.

score_1:0.672


In [71]:
y_pred_w_fselect = model_4.predict(X_test_reduced)#y_prediction with feature selection

In [72]:
y_pred_without_fs = model_1.predict(test_vector)#y_prediction without doing feature selection

In [67]:
#Output Precision, Recall, and F1 score of all three decision trees on respective test splits
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(test_labels, y_pred_without_fs))
print(classification_report(test_labels, y_pred_without_fs))

[[149  90]
 [ 81 180]]
              precision    recall  f1-score   support

           0       0.65      0.62      0.64       239
           1       0.67      0.69      0.68       261

    accuracy                           0.66       500
   macro avg       0.66      0.66      0.66       500
weighted avg       0.66      0.66      0.66       500



In [68]:
#Output Precision, Recall, and F1 score of all three decision trees on respective test splits
print(confusion_matrix(test_labels, y_pred_w_fselect))
print(classification_report(test_labels, y_pred_w_fselect))

[[162  77]
 [ 72 189]]
              precision    recall  f1-score   support

           0       0.69      0.68      0.68       239
           1       0.71      0.72      0.72       261

    accuracy                           0.70       500
   macro avg       0.70      0.70      0.70       500
weighted avg       0.70      0.70      0.70       500



In [22]:
np.shape(test_array)

(500, 12366)

In [23]:
np.shape(train_array)

(2000, 25235)